## Index constituent analysis
This notebook demonstrates an high level analysis of index constituents using the Data Libraries, including:
- ingesting consitiuent fundamentals
- market movers
- events
- index joiners and leavers

#### Learn more

To learn more about the Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
this [Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The LSEG Developer Community will be happy to help. 


## Imports

In [1]:
import pandas as pd
import refinitiv.data as rd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

rd.open_session()

<refinitiv.data.session.Definition object at 0x1574330a0 {name='workspace'}>

Below we define the index for the analysis

In [2]:
index_ric = '.FTSE'

## Index profile and price performance

#### Overview

In [3]:
overview = rd.get_data(
        universe = index_ric, 
        fields = ["TR.ISIN","TR.IndexSeriesName", 
                "TR.IndexWeightingName", "TR.CommonName()",
                "TR.IndexGeography","TR.TRBCIndustryGroup", 
                "TR.IndexCalculationCurrency"
                ]
)
overview

,Instrument,ISIN,Index Series Name,Weighting Name,Company Common Name,Index Geography,TRBC Industry Group Name,Calculation Currency
0,.FTSE,GB0001383545,FTSE UK - All Share,MarketCapInclusion,FTSE International Ltd,United Kingdom,Investment Banking & Investment Services,GBP


#### Overall price performance

In [4]:
price_performance = rd.get_data(
        universe = index_ric, 
        fields = ["HST_CLOSE", "TURNOVER", "PCTCHG_YTD", 
                "PCTCHG_QTD", "PCTCHG_MTD", "TR.Index_PE_RTRS", 
                "TR.Index_DIV_YLD_RTRS", "TR.IndexMktCapVendor"
                ]
)
price_performance

,Instrument,Calculated PE Ratio,Calculated Index Dividend Yield,Index Market Cap,HST_CLOSE,TURNOVER,PCTCHG_YTD,PCTCHG_QTD,PCTCHG_MTD
0,.FTSE,14.82957,3.68855,2067543982167.070068,8254.18,75859.96,6.37,3.4357,1.0034


#### Historical price performance

In [5]:
prices = rd.get_history(
        universe = index_ric, 
        start='2015-01-01', 
        end = '2024-04-04'
)
prices

.FTSE,TRDPRC_1,OPEN_PRC,HIGH_1,LOW_1,ACVOL_UNS,TRNOVR_UNS,TRDVOL_1
Date,,,,,,,
2015-01-02,6547.8,6566.09,6607.89,6510.6,378930102,226129.0,8294176
2015-01-05,6417.16,6547.8,6576.74,6404.49,750516616,461466.0,5284058
2015-01-06,6366.51,6417.16,6452.66,6328.59,793260855,478865.0,19054873
2015-01-07,6419.83,6366.51,6459.74,6366.51,709496420,421903.0,3468023
2015-01-08,6569.96,6419.83,6580.82,6419.83,910041429,483943.0,4039235
...,...,...,...,...,...,...,...
2024-03-27,7931.98,7930.96,7938.14,7893.0,891541286,508651.14,3129868
2024-03-28,7952.62,7931.98,7975.38,7931.36,906583822,482471.92,1901875
2024-04-02,7935.09,7952.62,8015.63,7928.59,984935307,564805.05,4307448


In [6]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'), 
               row_width=[0.5, 1])

fig.add_trace(go.Candlestick(x=prices.index, open=prices["OPEN_PRC"], high=prices["HIGH_1"],
                low=prices["LOW_1"], close=prices["TRDPRC_1"], name="OHLC"), 
                row=1, col=1
)

fig.add_trace(go.Bar(x=prices.index, y=prices["TRDVOL_1"], name = 'Volume', 
                     marker_color = 'firebrick'), 
                     row=2, col=1)

fig.update(layout_xaxis_rangeslider_visible=False)
fig.update_layout(width=1000, height=500)

fig.show()

## Constituent Analysis

#### Summary

In [7]:
summary = rd.get_data(
        universe = f'0#{index_ric}', 
        fields = [
            "TRDPRC_1",  "TR.IndexConstituentWeightPercent", 
            "TR.CompanyMarketCap", "TR.PE","TR.PriceToSalesPerShare",
            "TR.F.PriceToBookValuePerShr","TR.EPSActValue", 
            "TR.DPSActValue","TR.ROEActValue",
            ]
)
summary

,Instrument,Weight percent,Company Market Cap,P/E (Daily Time Series Ratio),Price To Sales Per Share (Daily Time Series Ratio),Price to Book Value per Share,Earnings Per Share - Actual,Dividend Per Share - Actual,Return On Equity - Actual,TRDPRC_1
0,STAN.L,0.822004,19922771810.102001,8.907988,0.905321,0.466601,1.26,0.27,7.2,772.2
1,CRDA.L,0.308591,6397417019.73823,37.435516,3.77333,2.997488,1.674,1.09,9.82,4544.7448
2,ANTO.L,0.386524,22795655726.825199,33.170519,4.599134,2.354353,0.72,0.36,8.08,2310.0
3,EZJ.L,0.143209,3518338945.87129,9.629476,0.401875,1.162158,0.45,0.045,14.49,457.042
4,BNZL.L,0.486676,10092518696.7535,19.118156,0.855039,3.635125,1.899,0.683,22.517,2970.0
...,...,...,...,...,...,...,...,...,...,...
95,ULVR.L,5.105034,106534393669.759995,19.533409,2.101032,6.054144,2.6,1.71,35.49,4235.0
96,OCDO.L,0.120173,3408499048.26159,<NA>,1.205886,3.267498,-0.5032,0.0,-24.71,389.9
97,LSEG.L,2.146384,48734038969.854698,66.434783,5.813021,2.108276,3.221,1.15,7.128,9066.0
98,TSCO.L,1.046369,21778728876.029999,12.688119,0.319397,1.686907,0.2341,0.121,14.523,310.9


In [8]:
summary_top_10 = summary.sort_values(by = 'Weight percent', ascending=False)[:10]
summary_top_10

,Instrument,Weight percent,Company Market Cap,P/E (Daily Time Series Ratio),Price To Sales Per Share (Daily Time Series Ratio),Price to Book Value per Share,Earnings Per Share - Actual,Dividend Per Share - Actual,Return On Equity - Actual,TRDPRC_1
85,SHEL.L,8.73484,176852200585.119995,13.114629,0.746888,1.144387,4.155,1.295,14.984,2827.5
10,AZN.L,8.591155,186564632966.979004,37.86795,5.000025,5.343453,7.26,2.9,32.299,11964.0
11,HSBA.L,6.356423,129120793425.919998,7.582437,1.638372,0.929665,1.21,0.61,13.041,685.4
95,ULVR.L,5.105034,106534393669.759995,19.533409,2.101032,6.054144,2.6,1.71,35.49,4235.0
53,BP.L,3.926675,81431770738.270004,11.436245,0.51229,1.420758,0.8,0.28,20.079,494.45
50,GSK.L,3.383611,72161503503.229904,15.937715,2.346191,4.407274,1.551,0.58,52.48,1736.0
8,REL.L,3.139673,64149281733.360001,36.600218,7.002432,16.911817,1.14,0.588,59.59,3414.0
70,RIO.L,2.942428,95121752499.619202,11.574218,2.144022,2.210377,7.25,4.35,22.44,5537.0
86,GLEN.L,2.867479,59694234472.776901,18.442213,0.349677,1.678619,0.53,0.13,14.41,486.55
38,DGE.L,2.80896,58448545210.330002,18.972141,3.647599,9.744233,1.96089,0.9624,47.388,2613.5


#### Price Performance

In [9]:
price_performance = rd.get_data(
        universe = f'0#{index_ric}', 
        fields = [
            "TR.TotalReturn1Wk", "TR.TotalReturn1Mo",
            "TR.TotalReturn3Mo", "TR.TotalReturn6Mo",
            "TR.TotalReturn1Yr", "TR.PricePctChgMTD",
            "TR.PricePctChgYTD", 'TR.AvgDailyVolume5D',
            'TR.AvgDailyVolume30D','TR.AvgDailyVolume90D'
            ]
)
price_performance

,Instrument,1 Week Total Return,1 Month Total Return,3 Month Total Return,6 Month Total Return,1 Year Total Return,MTD Price PCT Change,YTD Price PCT Change,Average Daily Volume - 5 Days,Average Daily Volume - 30 Days,Average Daily Volume - 90 Days
0,STAN.L,-0.590652,13.61902,22.197863,19.798931,22.92352,12.235431,16.141614,5493526,8134652,10530697
1,CRDA.L,-2.822581,-2.636615,0.816255,3.125367,-25.023729,-0.672451,-9.326733,323186,430767,565746
2,ANTO.L,-4.107884,2.48337,31.430598,64.033896,68.837437,4.570136,37.600476,995049,1356613,1598049
3,EZJ.L,-0.2151,-13.805277,-14.409594,11.029104,-3.655833,-13.869291,-9.039216,4721903,6673118,6277719
4,BNZL.L,-1.777485,-1.201548,-3.250001,1.953361,-4.444882,-2.927781,-6.45768,578509,672306,755242
...,...,...,...,...,...,...,...,...,...,...,...
95,ULVR.L,-0.514259,3.860004,10.632319,14.412867,5.850094,2.801932,12.0,3857968,4122049,4609930
96,OCDO.L,15.605634,16.227698,-16.364377,-27.619048,0.835381,16.227698,-45.886076,4771306,4559363,3895863
97,LSEG.L,-1.904558,2.619208,3.775341,3.752058,9.492472,3.640063,-1.14298,728704,1329987,1765123
98,TSCO.L,-0.256492,10.580947,15.206764,12.568131,22.772416,4.994938,7.091222,26143930,20788113,22548483


#### Volatility

In [10]:
volatility = rd.get_data(
        universe = f'0#{index_ric}', 
        fields = [
            "TR.Volatility10D", "TR.Volatility30D",
            "TR.Volatility60D", "TR.Volatility90D",
            "TR.Volatility200D", "TR.WACCBeta"
                ]
)
volatility

,Instrument,Volatility - 10 days,Volatility - 30 days,Volatility - 60 days,Volatility - 90 days,Volatility - 200 days,Beta
0,STAN.L,10.65384,26.493641,28.130721,27.797827,28.852709,1.456761
1,CRDA.L,19.409076,23.210671,25.091594,28.685904,29.630441,0.760414
2,ANTO.L,40.665171,31.961755,32.561952,32.174602,31.315437,1.176063
3,EZJ.L,34.146492,32.186616,32.337601,30.673176,32.646731,3.050846
4,BNZL.L,10.732367,13.878798,14.525669,14.744076,15.023901,0.628165
...,...,...,...,...,...,...,...
95,ULVR.L,8.989001,18.287418,16.947402,16.728134,14.506789,0.320905
96,OCDO.L,59.161529,50.489932,51.668967,49.529365,60.550566,1.299887
97,LSEG.L,12.879972,16.346364,14.787376,13.575322,12.54045,0.454842
98,TSCO.L,13.285629,15.323463,19.384572,19.886632,18.141857,0.651458


#### Relative Strength

In [11]:
rel_strength = rd.get_data(
        universe = f'0#{index_ric}', 
        fields = [
            "TR.RSIWilder3D", "TR.RSIWilder9D",
            "TR.RSIWilder14D", "TR.RSIWilder30D",
            "TR.PricePctChg52WkHigh", "TR.PricePctChg52WkLow",
            "TR.PriceAvg30D", "TR.PriceAvg60D",
            "TR.Price200DayAverage"
            ]
)
rel_strength

,Instrument,RSI Wilder Smoothing - 3 Days,RSI Wilder Smoothing - 9 Days,RSI Wilder Smoothing - 14 Days,RSI Wilder Smoothing - 30 Days,52-week High Price PCT Change,52-week Low Price PCT Change,30-day SMA,60-day SMA,200-day SMA
0,STAN.L,32.661791,62.37551,66.111308,64.139853,-2.738693,36.063269,735.686667,703.506667,676.652
1,CRDA.L,17.048264,31.731258,37.89722,44.624862,-27.155584,13.96217,4750.133333,4760.0,4766.505
2,ANTO.L,67.700691,56.898375,57.424539,59.562888,-4.701031,80.546875,2248.033333,2129.758333,1700.44
3,EZJ.L,50.097795,27.251688,30.150324,38.087864,-21.514953,32.542857,513.893333,530.493333,483.9845
4,BNZL.L,0.720994,23.51578,34.086323,42.947604,-9.743525,11.343284,3067.933333,3043.35,3025.26
...,...,...,...,...,...,...,...,...,...,...
95,ULVR.L,21.273096,52.739174,59.933548,61.437226,-2.318109,15.919924,4152.533333,4011.225,3941.58
96,OCDO.L,93.391443,74.431785,65.400718,51.497577,-59.646018,23.354373,358.11,394.983333,553.6355
97,LSEG.L,13.441991,44.164532,47.962522,49.967521,-5.581874,17.780062,9114.933333,9239.966667,8831.18
98,TSCO.L,30.52027,56.515354,59.898245,59.124922,-1.581778,43.015414,302.056667,294.991667,283.414


#### Gearing

In [13]:
gearing = rd.get_data(
        universe = f'0#{index_ric}', 
        fields = [
            "TR.F.QuickRatio","TR.F.CurrRatio", "TimesInterestEarned",
            "TR.F.LTDebtPctofTotCap","TR.F.LTDebtPctofComEq", 
            "TR.F.TotLiabPctofTotAssets", "TR.F.TotLiabPctofTotEq"
        ]
)
gearing

,Instrument,Quick Ratio,Current Ratio,Long Term Debt Percentage of Total Capital,Long Term Debt Percentage of Common Equity,Total Liabilities Percentage of Total Assets,Total Liabilities Percentage of Total Equity
0,STAN.L,<NA>,<NA>,40.692,159.38757,93.88061,1534.15089
1,CRDA.L,1.77452,2.8401,21.43136,28.04251,33.83717,51.14227
2,ANTO.L,2.06472,2.37121,19.70101,35.49302,38.67625,63.06896
3,EZJ.L,<NA>,0.99662,39.39341,80.15788,71.67971,253.1037
4,BNZL.L,0.91373,1.4034,31.8815,65.04737,66.08858,194.88588
...,...,...,...,...,...,...,...
95,ULVR.L,0.5438,0.76156,48.0013,130.88057,72.41251,262.48314
96,OCDO.L,2.27184,2.4922,54.86761,128.0355,65.88395,193.11714
97,LSEG.L,<NA>,1.00138,21.13458,31.64195,96.77718,3002.87542
98,TSCO.L,0.46352,0.6609,47.99291,108.99666,75.20143,303.24904


#### Management Effectiveness

In [14]:
management_effect = rd.get_data(
          universe = f'0#{index_ric}',
          fields = [          
               'TR.F.ReturnAvgComEqPct',
               'TR.F.ReturnAvgComEqPct5YrAvg',
               'TR.F.ReturnAvgTotAssetsPct',
               'TR.F.ReturnAvgTotAssetsPct5YrAvg'
          ] 
)
management_effect

,Instrument,Return on Average Common Equity - %,"Return on Average Common Equity - %, 5 Yr Avg",Return on Average Total Assets - %,"Return on Average Total Assets - %, 5 Yr Avg"
0,STAN.L,6.244114,3.877236,0.421484,0.300742
1,CRDA.L,7.172668,18.911725,4.786472,10.799385
2,ANTO.L,9.97645,11.67339,7.080281,8.684757
3,EZJ.L,12.180451,-10.967396,3.192905,-3.164753
4,BNZL.L,18.504712,20.35609,6.043413,6.22892
...,...,...,...,...,...
95,ULVR.L,34.948684,39.558583,9.328029,9.652699
96,OCDO.L,-18.885514,-18.465799,-8.374629,-7.308974
97,LSEG.L,3.056041,3.867741,0.115598,0.081435
98,TSCO.L,14.723721,7.895795,3.797346,2.115765


#### Fundamental

In [15]:
fundamentals = rd.get_data(
          universe  = f'0#{index_ric}',
          fields = [  
               'TR.Revenue',   
               'TR.F.EBITDA',
               'TR.F.NetIncAfterTax',
               "TR.EPSActValue",
               "TR.DPSActValue"
               'TR.FCFActValue',
               'TR.F.DebtTot',
               'TR.F.TotShHoldEq',
               'TR.F.EV',
               'TR.PCFullTimeEmployee',
          ]
)
fundamentals

,Instrument,Revenue,Earnings before Interest Taxes Depreciation & Amortization,Net Income after Tax,Earnings Per Share - Actual,Debt - Total,Total Shareholders' Equity incl Minority Intr & Hybrid Debt,Enterprise Value,Full-Time Employees (Pvt)
0,STAN.L,<NA>,6738000000.0,3462000000.0,1.26,139473000000,50353000000,-66060702006.713303,84958
1,CRDA.L,1694500000,409500000.0,172100000.0,1.674,710100000,2368100000,7604789821.0,5852
2,ANTO.L,6324500000,3032600000.0,1354700000.0,0.72,4079200000,12048400000,24875032323.321999,29000
3,EZJ.L,8171000000,1146000000.0,324000000.0,0.45,2884000000,2787000000,3197976836.825,<NA>
4,BNZL.L,11797100000,1146700000.0,526200000.0,1.899,3085800000,2966300000,12441078989.700001,24528
...,...,...,...,...,...,...,...,...,...
95,ULVR.L,59604000000,11510000000.0,7140000000.0,2.6,28593000000,20764000000,134932601661.317993,128000
96,OCDO.L,2825000000,-111600000.0,-387000000.0,-0.5032,1959900000,1511000000,5977494195.832,18869
97,LSEG.L,8379000000,3771000000.0,948000000.0,3.221,9699000000,25944000000,60317382589.559998,25608
98,TSCO.L,68187000000,4449000000.0,1758000000.0,0.2341,14841000000,11665000000,29848888440.68,<NA>


#### Dividents

In [ ]:
dividends = rd.get_data(
          universe = f'0#{index_ric}',
          fields = [  
               'TR.DividendYield',
               'TR.F.DivYldComStockIssuePctAnnized',
               'TR.DPSGrowth5Y'
               'TR.DivExDate',
               'TR.DivPayDate',
          ]
)
dividends

## Market Movers

#### Ingest constituent market cap, return and volume to produce a top mover table

In [18]:
market_mover = rd.get_data(
        universe = f'0#{index_ric}', 
        fields= [
            "TR.CompanyMarketCapitalization", 
            "TR.TotalReturn"]
)
market_mover

,Instrument,Company Market Capitalization,Total Return
0,STAN.L,20147863273.0508,-0.462844
1,CRDA.L,6444310223.58307,-1.080147
2,ANTO.L,23125178226.009201,2.894034
3,EZJ.L,3531444805.64024,1.398907
4,BNZL.L,10233102219.8265,-1.257445
...,...,...,...
95,ULVR.L,107833482074.332001,-0.931099
96,OCDO.L,3428115930.00121,9.791332
97,LSEG.L,49491667037.5243,-1.673102
98,TSCO.L,22182095144.534801,-1.238095


In [20]:
market_mover_sorted = market_mover.sort_values(by = "Total Return", ascending=False)
top_movers = pd.concat([market_mover_sorted[:10], market_mover_sorted[-10:]])
top_movers

,Instrument,Company Market Capitalization,Total Return
96,OCDO.L,3428115930.00121,9.791332
54,JD.L,6832971039.91998,5.434335
92,ICGIN.L,7077494843.53975,3.189655
14,FRES.L,4696105543.37937,3.020134
2,ANTO.L,23125178226.009201,2.894034
99,LGEN.L,15304074152.441401,1.976603
49,RS1R.L,3541631908.35899,1.755571
79,LLOY.L,35509750023.310303,1.423358
3,EZJ.L,3531444805.64024,1.398907
72,RR.L,38467158453.828598,1.393885


#### Plot total returns of top movers

In [21]:
categories = top_movers['Instrument']
values = top_movers['Total Return']
colors = ['green' if x > 0 else 'red' for x in values]

fig = go.Figure(data=[go.Bar(y=categories,x=values,marker_color=colors,orientation='h')])

fig.update_layout(title_text='Total Returns by company',
                  xaxis_title="Total Return Sum",
                  yaxis_title="Instrument",
                  plot_bgcolor="white",
                  width=1000, height=500)

fig.show()

#### Plot a treemap with tile size of market cap and color of total return

In [22]:
max_abs_return = max(top_movers['Total Return'].abs())
top_movers['Color'] = top_movers['Total Return'] / max_abs_return

colorscale = [
    [0, "red"],   
    [0.5, "white"],
    [1, "green"]
]

fig = go.Figure(go.Treemap(
    labels=top_movers['Instrument'],
    parents=[""]*len(top_movers),
    values=top_movers['Company Market Capitalization'],
    textinfo="label+value",
    hoverinfo="label+value+text",
    hovertext=top_movers['Total Return'],
    marker=dict(
        colors=top_movers['Color'],
        colorscale=colorscale,
        cmid=0 
    )
))

fig.update_layout(margin=dict(t=50, l=25, r=25, b=25), width=1000, height=500, 
                  title = 'Treemap for Top movers (tile size - Market Cap, color - Return)')
fig.show()

## Events

Below is a comprehensive list of event types

In [23]:
event_types_map = {'Brokerage analyst calls': 'BACALL',
                   'Brokerage analyst meetings': 'BAM',
                   'Company visits': 'CV',
                   'Conference presentations': 'CONFP',
                   'Conferences': 'CONF',
                   'Corporate analyst meetings': 'CAM',
                   'Corporate calls and presentations': 'CCALL',
                   'Earnings calls and presentations': 'ECALL',
                   'Earnings releases': 'RES',
                   'Ex Dividends': 'EXDIV',
                   'General & political events': 'GPE',
                   'Guidance calls and presentations': 'GCALL',
                   'IPO filings': 'IPOF',
                   'IPO lockup expiration': 'IPOLE',
                   'IPO pricings': 'IPOP',
                   'IPO withdrawals': 'IPOW',
                   'Market holidays': 'MH',
                   'Merger & Acquisition calls and presentations': 'MACALL',
                   'Other brokerages': 'OTHB',
                   'Other Corporate events': 'OTHC',
                   'Sales & Trading statement calls and presentations': 'STCALL',
                   'Sales & Trading statement releases': 'STR',
                   'Secondary filings': 'SECF',
                   'Secondary pricings': 'SECP',
                   'Secondary withdrawals': 'SECW',
                   'Shareholder / Annual meetings': 'SHM',
                   'Stock splits': 'SSP',
                   'Syndicate roadshows': 'SYNR'}

Let's now request several events from the constituents during the provided period.

In [24]:
event_types  = 'RES; EXDIV'
start = '2024-01-01'
end = '2025-01-01'

events_df = rd.get_data(
        universe = f'0#{index_ric}', 
        fields = [
            'TR.EventStartDate',
            'TR.EventType', 
            'TR.EventTitle'
            ],
        parameters = {'SDate': start, 'EDate': end, 'EventType': event_types})
events_df

,Instrument,Event Start Date,Company Event Type,Event Title
0,STAN.L,2024-01-12,ExDividends,US853254AB69=TRU Final Cash Dividend of gross ...
1,STAN.L,2024-01-12,ExDividends,US853254AB69=TRE Final Cash Dividend of gross ...
2,STAN.L,2024-01-12,ExDividends,37ZL.TWEA Final Cash Dividend of gross USD 350...
3,STAN.L,2024-01-12,ExDividends,US853254AC43=TRU Final Cash Dividend of gross ...
4,STAN.L,2024-01-12,ExDividends,US853254AC43=TRE Final Cash Dividend of gross ...
...,...,...,...,...
2014,ULVR.L,2024-02-22,ExDividends,UL.BA^E24 Interim Cash Dividend of gross USD 0...
2015,OCDO.L,2024-02-29,EarningsReleases,Full Year 2023 Ocado Group PLC Earnings Release
2016,LSEG.L,2024-02-29,EarningsReleases,Full Year 2023 London Stock Exchange Group PLC...
2017,TSCO.L,NaT,,


## Constituent changes

#### Request index joiners and leavers

In [26]:
start = "2015-01-01"
end = "2024-05-20"
const_changes  = rd.get_data(
        universe=index_ric, 
        fields = [
            "TR.IndexJLConstituentRIC.date", "TR.IndexJLConstituentRIC",
            "TR.IndexJLConstituentName", "TR.IndexJLConstituentRIC.change"
            ],
            parameters={"SDATE":start,"EDATE":end, 'IC':'B'}
)
const_changes

,Instrument,Date,Constituent RIC,Constituent Name,Change
0,.FTSE,2015-02-11,TUIJe.L^B15,TUI,Leaver
1,.FTSE,2015-03-23,HIK.L,Hikma Pharma,Joiner
2,.FTSE,2015-03-23,TLW.L,Tullow,Leaver
3,.FTSE,2015-04-13,MERL.L^K19,Merlin Ent,Joiner
4,.FTSE,2015-04-13,FLG.L^D15,Friends Life,Leaver
...,...,...,...,...,...
191,.FTSE,2023-12-18,HRGV.L,Hargreaves,Leaver
192,.FTSE,2024-01-16,PSN.L,Persimmon,Joiner
193,.FTSE,2024-01-16,DPH.L^A24,Dechra Pharms,Leaver
194,.FTSE,2024-03-18,EZJ.L,easyJet,Joiner


#### Group joiner/leaver list by year and plot the index turnover

In [27]:
const_changes_grouped = const_changes.groupby(by = 'Date').count().reset_index()
const_changes_grouped['Year'] = const_changes_grouped['Date'].dt.year
const_changes_grouped

,Date,Instrument,Constituent RIC,Constituent Name,Change,Year
0,2015-02-11,1,1,1,1,2015
1,2015-03-23,2,2,2,2,2015
2,2015-04-13,2,2,2,2,2015
3,2015-06-22,2,2,2,2,2015
4,2015-09-21,2,2,2,2,2015
...,...,...,...,...,...,...
62,2023-09-18,8,8,8,8,2023
63,2023-09-25,2,2,2,2,2023
64,2023-12-18,2,2,2,2,2023
65,2024-01-16,2,2,2,2,2024


In [28]:
fig = go.Figure(data=[go.Bar(x=const_changes_grouped['Year'], y=const_changes_grouped['Change'])])


fig.update_layout(
    title_text='Total number of Index Joiner/Leavers each year',
    xaxis=dict(
        type='category',
        categoryorder='array',
        categoryarray=sorted(const_changes_grouped['Year'].unique())
    ),
    width=1000, height=500
)

fig.show()

In [29]:
rd.close_session()